In [1]:
import advertools as adv
import adviz
import pandas as pd
pd.options.display.max_columns = None
pd.set_option('future.no_silent_downcasting', True)

## Read the csv file exported from BigQuery

In [2]:
gsc = pd.read_csv('gsc_big_query_export_url.csv')

## Remove all `is_<search_appearance_type>` columns that contain all `False` values

In [3]:
drop_bools = gsc.select_dtypes(bool).columns[gsc.select_dtypes(bool).sum().eq(0)]
gsc = gsc.drop(drop_bools, axis=1)

## Convert `data_date` to datetime type

In [4]:
gsc['data_date'] = pd.to_datetime(gsc['data_date'], format='%d/%m/%Y')

## Add country flags

In [5]:
gsc['flag'] = [adviz.flag(x) for x in gsc['country']]

## Split URLs and combine relevant columns with original DataFrame

In [6]:
urldf = adv.url_to_df(gsc['url'])
gsc = pd.concat([gsc, urldf.filter(regex=r'^dir_\d')], axis=1)
gsc['dir_1'] = gsc['dir_1'].fillna('/')

In [7]:
gsc.head()

,data_date,site_url,url,query,is_anonymized_query,country,search_type,device,impressions,clicks,sum_position,flag,dir_1,dir_2
0,2024-07-05,https://nbastats.pro/,https://nbastats.pro/,NaN,True,are,WEB,MOBILE,1,0,0,🇦🇪,/,None
1,2024-07-05,https://nbastats.pro/,https://nbastats.pro/player/Larry_Bird,NaN,True,usa,WEB,DESKTOP,1,0,13,🇺🇸,player,Larry_Bird
2,2024-07-05,https://nbastats.pro/,https://nbastats.pro/player/Robin_Jones,NaN,True,are,WEB,MOBILE,1,0,5,🇦🇪,player,Robin_Jones
3,2024-07-05,https://nbastats.pro/,https://nbastats.pro/player/Rick_Fox,NaN,True,are,WEB,MOBILE,1,0,10,🇦🇪,player,Rick_Fox
4,2024-07-05,https://nbastats.pro/,https://nbastats.pro/player/Tony_Robertson,NaN,True,are,WEB,MOBILE,1,0,8,🇦🇪,player,Tony_Robertson


## Create query categories and regexes to match them

In [8]:
query_cat = {
    'basketball': r'basketbal',
    'stats': r'statistics|stats',
    'career': r'career',
}

## Create `cat_` columns for each of the created categories

In [9]:
for cat, regex in query_cat.items():
    gsc[f'cat_{cat}'] = gsc['query'].fillna('').str.contains(regex, regex=True)

In [10]:
gsc.dropna(subset=['query']).drop(['site_url', 'is_anonymized_query'], axis=1).sample(15).style.background_gradient(subset=gsc.filter(regex='cat_').columns.tolist() + ['impressions', 'clicks', 'sum_position'], cmap='cividis')


,data_date,url,query,country,search_type,device,impressions,clicks,sum_position,flag,dir_1,dir_2,cat_basketball,cat_stats,cat_career
220,2024-09-05 00:00:00,https://nbastats.pro/player/Luol_Deng,luol deng stats,fin,WEB,DESKTOP,1,0,23,🇫🇮,player,Luol_Deng,False,True,False
52,2024-07-05 00:00:00,https://nbastats.pro/player/Chuck_Cooper,chuck cooper stats,gbr,WEB,DESKTOP,1,0,24,🇬🇧,player,Chuck_Cooper,False,True,False
48,2024-07-05 00:00:00,https://nbastats.pro/player/Larry_Bird,bird career stats,bhs,WEB,MOBILE,1,0,11,🇧🇸,player,Larry_Bird,False,True,True
274,2024-09-05 00:00:00,https://nbastats.pro/player/Larry_Bird,larry bord stats,usa,WEB,DESKTOP,1,0,17,🇺🇸,player,Larry_Bird,False,True,False
101,2024-08-05 00:00:00,https://nbastats.pro/player/Larry_Bird,larry bird stats,rou,WEB,DESKTOP,1,0,22,🇷🇴,player,Larry_Bird,False,True,False
93,2024-08-05 00:00:00,https://nbastats.pro/player/Jawann_Oldham,jawann oldham stats,gbr,WEB,DESKTOP,1,0,11,🇬🇧,player,Jawann_Oldham,False,True,False
270,2024-09-05 00:00:00,https://nbastats.pro/player/Kevin_Huerter,kevin huerter stats,usa,WEB,MOBILE,1,0,38,🇺🇸,player,Kevin_Huerter,False,True,False
151,2024-08-05 00:00:00,https://nbastats.pro/player/Larry_Bird,larry bird stats,usa,WEB,MOBILE,11,0,135,🇺🇸,player,Larry_Bird,False,True,False
282,2024-09-05 00:00:00,https://nbastats.pro/player/Larry_Bird,larry bird career statistics,usa,WEB,MOBILE,1,0,20,🇺🇸,player,Larry_Bird,False,True,True
231,2024-09-05 00:00:00,https://nbastats.pro/player/Jeff_Turner,jeff turner stats,gbr,WEB,DESKTOP,1,1,19,🇬🇧,player,Jeff_Turner,False,True,False
